In [96]:
from utils import increment_path

import os
from IPython.lib.display import Audio
import numpy as np

# ********** CHANGE HERE (1/2): Name for the path of reader model to be saved **********
exp_name = "xlm_roberta_fine_tuning_continue"
# **************************************************************************************
model_output_path = increment_path("/opt/ml/outputs/models", "/no", exp_name)
print(model_output_path)

# ********** CHANGE HERE (2/2): Path of state of reader model if needed, else "no" (default). **********
model_state_path = "no"
#model_state_path = "/opt/ml/outputs/models/no2_xlm_roberta_state_transfer_learning2/xlm-roberta-large_state_dict.pth"
# ******************************************************************************************************
# seed == 42 (default)
# ********** CHANGE with parser: train_korquad / save_state_only / num_train_epochs / learning_rate(transfer-learning > fine-tuning)
model_path = "/opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue/checkpoint-1176"
!python train.py\
    --model_path $model_path\
    --train_korquad False\
    --save_state_only False\
    --model_state_path $model_state_path\
    \
    --output_dir $model_output_path\
    --fp16 True --fp16_opt_level "O1"\
    --dataloader_pin_memory True\
    --dataloader_drop_last False\
    \
    --num_train_epochs 15.0\
    --per_device_train_batch_size 4 --gradient_accumulation_steps 4\
    --save_strategy "epoch"\
    --save_total_limit 1\
    \
    --learning_rate 5e-5 --lr_scheduler_type "constant"\
    --warmup_ratio 0.1\
    --weight_decay 0.001 --adam_beta1 0.9 --adam_beta2 0.999 --adam_epsilon 1e-8\
    --adafactor False\
    --max_grad_norm 1.0\
    \
    --group_by_length False\
    --label_smoothing_factor 0.0

print("Training End.")

# loop_inference.ipynb를 위해 EOI 폴더를 만듦
if os.path.isdir(model_output_path):
    eof_dir_path = model_output_path + "/zeof"
    os.mkdir(eof_dir_path)

# 종료를 알리는 소리
def beep():
    framerate = 4410
    play_time_seconds = 2

    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
    audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
    return Audio(audio_data, rate=framerate, autoplay=True)
beep()

/opt/ml/outputs/models/no9_xlm_roberta_fine_tuning_continue
data is from /opt/ml/input/data/train_dataset
model is from /opt/ml/outputs/models/no8_xlm_roberta_fine_tuning_continue/checkpoint-1176
model uses device: cuda:0
DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 91.99ba/s]
train dataset:
 Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 4192
})
100%|█████████████████████████████████████████████| 5/5 [00:05<00:00,  1.08s/ba]
Init Trainer...
  0%|                                                  | 0/8820 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: Us

OSError: [Errno 28] No space left on device: '/opt/ml/outputs/models/no9_xlm_roberta_fine_tuning_continue/zeof'

In [99]:
from utils import increment_path

exp_name = "KorBERT"
test_output_dir = increment_path("/opt/ml/outputs/preds", "/no", exp_name)

dataset_path = "/opt/ml/input/data/test_dataset"
model_path = "/opt/ml/outputs/models/no3_xlm_roberta_fine_tuning_tl1/checkpoint-882"
!python inference.py\
    --output_dir $test_output_dir\
    --topk 20

print("Inference End.")
#--model_path $model_path\

# 종료를 알리는 소리
def beep():
    framerate = 4410
    play_time_seconds = 2

    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
    audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
    return Audio(audio_data, rate=framerate, autoplay=True)
beep()

data is from /opt/ml/input/data/test_dataset
model is from xlm-roberta-large
Dataset({
    features: ['id', 'question'],
    num_rows: 600
})
Lengths of unique contexts : 56607
Passage Embedding Loaded.
[Relevant documents exhaustive search.] done in 5.699 s
Making DataFrame dataset:: 600it [00:00, 17431.36it/s]
Traceback (most recent call last):
  File "inference.py", line 117, in <module>
    main()
  File "inference.py", line 77, in main
    load_from_cache_file=not data_args.overwrite_cache,
  File "/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py", line 1407, in map
    update_data = does_function_return_dict(test_inputs, test_indices)
  File "/opt/conda/lib/python3.7/site-packages/datasets/arrow_dataset.py", line 1378, in does_function_return_dict
    function(*fn_args, indices, **fn_kwargs) if with_indices else function(*fn_args, **fn_kwargs)
  File "inference.py", line 73, in <lambda>
    lambda example: prepare_validation_features(example, tokenizer, data_args)

In [216]:
eval_output_dir = increment_path("./outputs/train_dataset", "/exp", exp_name)
!python train.py\
    --do_eval\
    --output_dir $eval_output_dir\
    --model_name_or_path $model_output_dir

In [3]:
!python train.py --do_train\
    --run_name None\
    --output_dir ./models/train_dataset --overwrite_output_dir False\
    --seed 42\
    --fp16 True --fp16_opt_level "O1"\
    --dataloader_num_workers 4\
    --dataloader_pin_memory True\
    --dataloader_drop_last False\
    \
    --num_train_epochs 10.0\
    --per_device_train_batch_size 16 --gradient_accumulation_steps 1\
    --per_device_eval_batch_size 16 -- eval_accumulation_steps 1\
    --evaluation_strategy "epoch"\
    --logging_strategy "steps" --logging_steps 500\
    --save_strategy "epoch"\
    --save_steps 500 --save_total_limit 3\
    --load_best_model_at_end False\
    \
    --learning_rate 5e-5 --lr_scheduler_type "cosine"\
    --warmup_ratio 0.0 --warmup_steps 0\
    --weight_decay 0.01 --adam_beta1 0.9 --adam_beta2 0.999 --adam_epsilon 1e-8\
    --adafactor False\
    --max_grad_norm 1.0\
    \
    --group_by_length False\
    --label_smoothing_factor 0.0

Traceback (most recent call last):
  File "train.py", line 348, in <module>
    main()
  File "train.py", line 34, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "/opt/conda/lib/python3.7/site-packages/transformers/hf_argparser.py", line 196, in parse_args_into_dataclasses
    raise ValueError(f"Some specified arguments are not used by the HfArgumentParser: {remaining_args}")
ValueError: Some specified arguments are not used by the HfArgumentParser: ['--', 'eval_accumulation_steps', '1', '--evaluation_strategy', 'epoch', '--logging_strategy', 'steps', '--logging_steps', '500', '--save_strategy', 'epoch', '--save_steps', '500', '--save_total_limit', '3', '--load_best_model_at_end', 'False', '--learning_rate', '5e-5', '--lr_scheduler_type', 'constant_with_warmup', '--warmup_ratio', '0.0', '--warmup_steps', '0', '--weight_decay', '0', '--adam_beta1', '0.9', '--adam_beta2', '0.999', '--adam_epsilon', '1e-8', '--adafactor', 'False', '--max_gra

In [ ]:
# class EvaluationStrategy, IntervalStrategy(ExplicitEnum):
    NO = "no"
    STEPS = "steps"
    EPOCH = "epoch"

# class SchedulerType(ExplicitEnum):
    LINEAR = "linear"
    COSINE = "cosine"
    COSINE_WITH_RESTARTS = "cosine_with_restarts"
    POLYNOMIAL = "polynomial"
    CONSTANT = "constant"
    CONSTANT_WITH_WARMUP = "constant_with_warmup"

# fp16_opt_level: https://nvidia.github.io/apex/amp.html